# This notebook contains some experiments using sgp4 and h3

In [2]:
!pip install -r requirements.txt

In [3]:
from sgp4.api import Satrec
import julian
from datetime import datetime
from datetime import timedelta
import requests as r
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
from astropy.coordinates import ITRS
from astropy.time import Time
from h3 import h3
import folium

Get the orbital parameters for the starlink constellation

In [4]:
res = r.get("https://www.celestrak.com/NORAD/elements/starlink.txt")

Group every 3 linkes together (Two Line Element and satellite name)

In [5]:
sats = list(zip(*[iter(res.text.split('\r\n'))]*3))

This function calculates the position of a sattelite using the sgp4 model and converts the latitude+longitude format

In [6]:
def getSatPos(i, dt):
    """
    Returns cartesian location of a satellite at the given time
    
    Based on this:
    https://docs.astropy.org/en/stable/coordinates/satellites.html
    """
    
    sat = Satrec.twoline2rv(sats[i][1], sats[i][2])
    name = sats[i][0].strip()
    
    t = Time(dt)
    
    e, p, v = sat.sgp4(t.jd1, t.jd2)
    
    # TODO: make sure that e==0
    
    p = CartesianRepresentation(p*u.km)
    v = CartesianDifferential(v*u.km/u.s)
    
    teme = TEME(p.with_differentials(v), obstime=t)
    itrs = teme.transform_to(ITRS(obstime=t))
    
    location = itrs.earth_location
    
    #print(f"Satellite {name} is at {location.geodetic.lat} {location.geodetic.lon} at height {location.geodetic.height}")
    
    return location

This function shows the tiles on a map

In [7]:
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    copied from 
    
    
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=2, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=4,color=color)
        m.add_child(my_PolyLine)
    return m

# Get the h3 tile over which a satellite currently is

In [50]:
dt = datetime(2021, 5, 7, 12, 4, 46)

In [51]:
location = getSatPos(1, dt)

In [52]:
h3_address = h3.geo_to_h3(location.geodetic.lat/u.deg, location.geodetic.lon/u.deg, 0) # lat, lng, hex resolution  
print(h3_address)
m = visualize_hexagons([h3_address])
display(m)

8045fffffffffff


# Get the h3 tile over which a satellite will cross in the next hour

In [53]:
locs = [getSatPos(1, dt+i*timedelta(minutes=1)) for i in range(60)]

In [54]:
h3locs = [h3.geo_to_h3(loc.geodetic.lat/u.deg, loc.geodetic.lon/u.deg, 0) for loc in locs]
m = visualize_hexagons(h3locs)
display(m)

# Visualize a whole LEO constellation

In [55]:
def satPos(sat, dt):
    """
    Returns cartesian location of a satellite at the given time
    
    Based on this:
    https://docs.astropy.org/en/stable/coordinates/satellites.html
    """
    
    sat = Satrec.twoline2rv(sat[1], sat[2])
    
    t = Time(dt)
    
    e, p, v = sat.sgp4(t.jd1, t.jd2)
    
    # TODO: make sure that e==0
    
    p = CartesianRepresentation(p*u.km)
    v = CartesianDifferential(v*u.km/u.s)
    
    teme = TEME(p.with_differentials(v), obstime=t)
    itrs = teme.transform_to(ITRS(obstime=t))
    
    location = itrs.earth_location
    
    #print(f"Satellite {name} is at {location.geodetic.lat} {location.geodetic.lon} at height {location.geodetic.height}")
    
    return location

In [56]:
res = r.get("https://www.celestrak.com/NORAD/elements/iridium-NEXT.txt")

In [57]:
sats = list(zip(*[iter(res.text.split('\r\n'))]*3))

In [58]:
locs = [satPos(sat, dt) for sat in sats]

In [59]:
h3locs = [h3.geo_to_h3(loc.geodetic.lat/u.deg, loc.geodetic.lon/u.deg, 0) for loc in locs]
m = visualize_hexagons(h3locs)
display(m)

# Initialize a LEO constellation (not fininshed yet)

In [ ]:
startJd, startFr = jday(2021,1,1,0,0,0)

In [ ]:
nSats = 10
nPlanes = 1

In [ ]:
sats = [Satrec()] * (nSats*nPlanes)
for plane in range(0, nPlanes):
    for sat in range(0, nSats):
        satNum = plane*nPlanes + sat
        sats[satNum].sgp4init(
            # gravity model
            WGS72,
            # 'a' = old AFSPC mode, 'i' = improved mod
            'i',
            # satnum: Satellite number
            satNum,
            # epoch: days since 1949 December 31 00:00 UT
            startJd - 2433281.5,
            # bstar: drag coefficient (1/earth radi
            -0.7438e-3,
            # ndot (NOT USED): ballistic coefficient (revs/day)
            0,
            # nddot (NOT USED): mean motion 2nd derivative (revs/day^3)
            0,
            # ecco: eccentricity
            0.0001205,
            # argpo: argument of perigee (radians)
            0,
            # inclo: inclination (radians)
            53.0,
            # mo: mean anomaly (radians)
            
            # no_kozai: mean motion (radians/minute)
            
            # nodeo: right ascension of ascending node (radians)
        
        )